# Примеры

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Пример 1, обучение RandomForest выявлению сепсиса

### Устройство MIMIC
В примерах разберем, как с помощью модуля sickness-screening обучить модель предсказывать сепсис на наборе данных MIMIC.
В MIMIC есть множество таблиц, но для примера нам потребуются следующие таблицы:
1. **chartevents.csv** —— содержит данные мониторинга пациентов, например: температура тела, артериальное давление.
2. **labevents.csv** —— содержит данные различных анализов пациентов, например различные характеристики анализов крови для пациентов.
3. **diagnoses.csv** —— содержит информацию о диагнозах, которые получил пациент.
4. **d_icd_diagnoses** —— расшифрофки кодов диагнозов для каждого диагноза.
5. **d_labitems.csv** —— расшифрофки кодов анализов для каждого пациента.

### Установим библиотеку

In [ ]:
!pip3 install sickness-screening

#### Аггрегирование данных о диагнозах пациентов:

##### Импортирование библиотеки

In [ ]:
import sickness_screening as ss

#### Аггрегирование данных о диагнозах пациентов:

In [ ]:
df_gotten_diagnoses = ss.get_diagnoses_data(patient_diagnoses_csv='diagnoses.csv',
                 all_diagnoses_csv='d_icd_diagnoses.csv',
                 output_file_csv='gottenDiagnoses.csv')
print(df_gotten_diagnoses.head(10))

   subject_id                                         long_title
0    10000032                                Portal hypertension
1    10000032                                      Other ascites
2    10000032      Cirrhosis of liver without mention of alcohol
3    10000032  Unspecified viral hepatitis C without hepatic ...
4    10000032  Chronic airway obstruction, not elsewhere clas...
5    10000032                      Bipolar disorder, unspecified
6    10000032                      Posttraumatic stress disorder
7    10000032                    Personal history of tobacco use
8    10000032    Unspecified viral hepatitis C with hepatic coma
9    10000032                                      Other ascites


Здесь мы для каждого пациента из **patient_diagnoses_csv** получаем коды диагнозов, а далее, используя **all_diagnoses_csv**
мы уже получаем файл **output_file_csv** в котором для каждого пациента уже хранится расшифровка его диагноза.

#### Получение данных о том, есть ли конкретный диагноз у пациента

In [ ]:
df_sepsis_info = ss.get_disease_info(diagnoses_csv='gottenDiagnoses.csv', title_column='long_title', disease_str='sepsis',
                    disease_column='has_sepsis', subject_id_column='subject_id', log_stats=True,
                    output_csv='sepsis_info.csv')
print(df_sepsis_info.head(20))

Total patients: 180640
Total patients with disease: 12616
    subject_id  has_sepsis
0     10000032       False
1     10000068       False
2     10000084       False
3     10000108       False
4     10000117       False
5     10000248       False
6     10000280       False
7     10000560       False
8     10000635       False
9     10000719       False
10    10000764       False
11    10000826        True
12    10000883       False
13    10000886       False
14    10000904       False
15    10000935       False
16    10000980       False
17    10001176       False
18    10001186       False
19    10001217       False


 Здесь используем таблицу, которую мы получили из предыдущего примера, чтобы на выходе получить таблицу, в которой содержатся данные о том,
был ли у этого человека в диагнозе продстрока sepsis, или нет.

#### Аггрегирование данных, необходимых для нахождения ССВР (синдром системной воспалительной рекции)

Теперь соберем некоторые данные, необходимые для определения ССВР:

In [ ]:
itemids_map = {
    220045: 'Heart rate',
    220210: 'Respiratory rate',
    223762: 'Temperature Fahrenheit',
    223761: 'Temperature Celsius',
    225651: 'Direct Bilirubin'
}
df_ssir = ss.get_analyzes_data(analyzes_csv='chartevents.csv', subject_id_col='subject_id', itemid_col='itemid',
                      charttime_col='charttime', value_col='value', valuenum_col='valuenum',
                      itemids_map=itemids_map, rest_columns=['Heart rate', 'Respiratory rate', 'Temperature Fahrenheit', 'Temperature Celsius',
                        'Direct Bilirubin', 'subject_id', 'charttime'], output_csv='ssir.csv')
print(df_ssir.head(10))

  Heart rate Heart rate Respiratory rate Respiratory rate  \
0        NaN        NaN              NaN              NaN   
1         91        bpm               24         insp/min   
2         93        bpm               21         insp/min   
3         94        bpm               23         insp/min   
4        105        bpm               21         insp/min   
5         97        bpm               20         insp/min   
6        100        bpm               21         insp/min   
7         97        bpm               16         insp/min   
8        100        bpm               19         insp/min   
9         94        bpm               22         insp/min   

  Temperature Fahrenheit Temperature Fahrenheit Temperature Celsius  \
0                    NaN                    NaN                98.7   
1                    NaN                    NaN                 NaN   
2                    NaN                    NaN                 NaN   
3                    NaN                    

Здесь мы используя таблицу **analyzes_csv**, **itemids**(коды анализов, которые мы хотим собрать), **rest_columns**(колонки, которые мы хотим оставить в выходной таблице),
Фунукция собирает из analyzes_csv замеры для пациентов с кодами **itemids** и записать их в **output_csv**, оставив только колонки, которые есть в **rest_columns**
В данной функции **subject_id_col** и **itemid_col** отвечают за колонки, отведенные под коды пациентов и анализов соответсвенно.
**charttime_col** отвечает за время. **valuenum_col** отвечает за колонку с единицами измерения анализов.

#### Комбинирование данных о диагнозах и ССВР

In [ ]:
df_combined = ss.combine_data(first_data_csv='gottenDiagnoses.csv',
                              second_data_csv='ssir.csv',
                              output_file='diagnoses_and_ssir.csv', log_stats=False)
print(df_combined.head(10))

   Heart rate Heart rate.1  Respiratory rate Respiratory rate.1  \
0         NaN          NaN               NaN                NaN   
1        91.0          bpm              24.0           insp/min   
2        93.0          bpm              21.0           insp/min   
3        94.0          bpm              23.0           insp/min   
4       105.0          bpm              21.0           insp/min   
5        97.0          bpm              20.0           insp/min   
6       100.0          bpm              21.0           insp/min   
7        97.0          bpm              16.0           insp/min   
8       100.0          bpm              19.0           insp/min   
9        94.0          bpm              22.0           insp/min   

   Temperature Fahrenheit Temperature Fahrenheit.1  Temperature Celsius  \
0                     NaN                      NaN                 98.7   
1                     NaN                      NaN                  NaN   
2                     NaN            

#### Сбор и комбинирование данных об анализах крови, с данными об диагнозах и ССВР
Соберем данные об анализах крови пациентов и скомбинируем их в одну таблицу:

In [ ]:
df_merged = ss.merge_and_get_data(merge_with='diagnoses_and_ssir.csv',
                      blood_csv='labevents.csv',
                      get_data_from='chartevents.csv',
                      disease_info_csv='sepsis_info_df.csv',
                      output_csv='merged_data.csv',
                      has_disease_column = 'has_sepsis',
                      analyzes_names = {
                        51222: "Hemoglobin",
                        51279: "Red Blood Cell",
                        51240: "Large Platelets",
                        50861: "Alanine Aminotransferase (ALT)",
                        50878: "Asparate Aminotransferase (AST)",
                        225651: "Direct Bilirubin",
                        50867: "Amylase",
                        51301: "White Blood Cells"})
print(df_merged.head(10))

/Users/artemij/miniconda3/lib/python3.12/site-packages/sickness_screening/merge_and_get_data.py:64: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  diagnoses_and_ssir = pd.read_csv(merge_with)


Unique patients with disease: 12615
Unique patients without disease: 163902
Total unique patients: 243811
   subject_id            charttime Alanine Aminotransferase (ALT) Amylase  \
0    10000032  2180-03-23 11:51:00                            102     NaN   
1    10000032  2180-05-06 22:25:00                            100     NaN   
2    10000032  2180-05-07 05:05:00                             88     NaN   
3    10000032  2180-06-03 12:00:00                            110     NaN   
4    10000032  2180-06-22 11:15:00                            113     NaN   
5    10000032  2180-06-26 16:10:00                            126     NaN   
6    10000032  2180-06-27 05:10:00                            117     NaN   
7    10000032  2180-07-23 06:39:00                            153     NaN   
8    10000032  2180-07-23 14:00:00                            NaN     NaN   
9    10000032  2180-07-23 14:12:00                            NaN     NaN   

  Asparate Aminotransferase (AST) Hemoglobin L

Данная функция ищет данные об **analyzes_names** пациентов из таблиц **blood_csv.csv** и **get_data_from**,
комбинирует их вместе с **merge_with**. Стоит отметить, что эта функция также комбинирует данные о болезни каждого пациента.

#### Балансировка данных внутри каждого пациента:

In [ ]:
df_balanced = ss.balance_on_patients(balancing_csv='merged_data.csv', disease_col='has_sepsis', subject_id_col='subject_id',
                        output_csv='balance.csv',
                        output_filtered_csv='balance_filtered.csv',
                        filtering_on=100,
                        number_of_patient_selected=70000,
                        log_stats=True
                        )
print(df_balanced.head(10))

/Users/artemij/miniconda3/lib/python3.12/site-packages/sickness_screening/balance_on_patients.py:29: DtypeWarning: Columns (3,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(balancing_csv)


In filtered dataset:
Unique patients with disease: 7302
Unique patients without disease: 7253
Number of lines for patients with disease: 2956816
Number of lines for patients without disease: 1748931

In original dataset
Number of lines for patients with disease: 3185562
Number of lines for patients without disease: 2832512
     subject_id            charttime Alanine Aminotransferase (ALT) Amylase  \
844    10002013  2155-07-27 13:26:00                             16     NaN   
845    10002013  2155-12-01 12:25:00                             16     NaN   
846    10002013  2156-03-04 11:25:00                             16     NaN   
847    10002013  2156-06-27 11:10:00                             17     NaN   
848    10002013  2156-06-28 14:07:00                            NaN     NaN   
849    10002013  2156-06-29 08:20:00                            NaN     NaN   
850    10002013  2156-10-22 11:27:00                             22     NaN   
851    10002013  2156-11-01 12:30:00       

#### Компрессия данных о каждом пациенте (если в наборе данных пропуски, то внутри каждого пациента пропуски заполнятся значением из этого пациента)
Теперь заполним пропуски имеющимися данными для каждого пациента, не заполняя статистическими значениями или константами:

In [ ]:
df_compressed = ss.compress(df_to_compress=df_balanced,
            subject_id_col='subject_id',
            output_csv='compressed_data.csv')
print(df_compressed.head(10))

/Users/artemij/miniconda3/lib/python3.12/site-packages/sickness_screening/compress.py:21: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda group: group.ffill().bfill().infer_objects(copy=False)


   subject_id            charttime Alanine Aminotransferase (ALT) Amylase  \
0    10002013  2155-07-27 13:26:00                             16      31   
1    10002013  2155-12-01 12:25:00                             16      31   
2    10002013  2156-03-04 11:25:00                             16      31   
3    10002013  2156-06-27 11:10:00                             17      31   
4    10002013  2156-06-28 14:07:00                             17      31   
5    10002013  2156-06-29 08:20:00                             17      31   
6    10002013  2156-10-22 11:27:00                             22      31   
7    10002013  2156-11-01 12:30:00                             22      31   
8    10002013  2156-11-02 06:30:00                             22      31   
9    10002013  2156-11-03 05:30:00                             22      31   

  Asparate Aminotransferase (AST) Hemoglobin Large Platelets Red Blood Cell  \
0                              15       12.3             NaN           3.

#### Выбрать лучших пациентов с данными для балансировки

In [ ]:
df_choose = ss.choose(compressed_df_csv='compressed_data.csv',
          output_file='final_balanced_data.csv')
print(df_choose.head(10))

/Users/artemij/miniconda3/lib/python3.12/site-packages/sickness_screening/choose.py:23: DtypeWarning: Columns (3,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(compressed_df_csv)


   subject_id            charttime Alanine Aminotransferase (ALT) Amylase  \
0    10002013  2155-07-27 13:26:00                             16    31.0   
1    10002013  2155-12-01 12:25:00                             16    31.0   
2    10002013  2156-03-04 11:25:00                             16    31.0   
3    10002013  2156-06-27 11:10:00                             17    31.0   
4    10002013  2156-06-28 14:07:00                             17    31.0   
5    10002013  2156-06-29 08:20:00                             17    31.0   
6    10002013  2156-10-22 11:27:00                             22    31.0   
7    10002013  2156-11-01 12:30:00                             22    31.0   
8    10002013  2156-11-02 06:30:00                             22    31.0   
9    10002013  2156-11-03 05:30:00                             22    31.0   

  Asparate Aminotransferase (AST) Hemoglobin Large Platelets Red Blood Cell  \
0                              15       12.3             NaN           3.

#### Заполнение пропущенных значений модой

In [ ]:
df_filled = ss.fill_values(balanced_csv='final_balanced_data.csv',
               strategy='most_frequent',
               output_csv='filled_data.csv')
print(df_filled.head(10))

/Users/artemij/miniconda3/lib/python3.12/site-packages/sickness_screening/fill_values.py:24: DtypeWarning: Columns (3,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(balanced_csv)


  subject_id            charttime Alanine Aminotransferase (ALT) Amylase  \
0   10002013  2155-07-27 13:26:00                             16    31.0   
1   10002013  2155-12-01 12:25:00                             16    31.0   
2   10002013  2156-03-04 11:25:00                             16    31.0   
3   10002013  2156-06-27 11:10:00                             17    31.0   
4   10002013  2156-06-28 14:07:00                             17    31.0   
5   10002013  2156-06-29 08:20:00                             17    31.0   
6   10002013  2156-10-22 11:27:00                             22    31.0   
7   10002013  2156-11-01 12:30:00                             22    31.0   
8   10002013  2156-11-02 06:30:00                             22    31.0   
9   10002013  2156-11-03 05:30:00                             22    31.0   

  Asparate Aminotransferase (AST) Hemoglobin Large Platelets Red Blood Cell  \
0                              15       12.3               A           3.64   
1    

#### Тренировка модели на наборе данных

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
def convert_white_blood_cells(x):
    if x == '<0.1':
        return 0
    else:
        return x
model = ss.train_model(df_to_train_csv='filled_data.csv',
                       categorical_cols=['Large Platelets'],
                       columns_to_train_on=['Amylase', 'Hemoglobin', 'Red Blood Cell', 'White Blood Cells'],
                       model=RandomForestClassifier(),
                       cat_convert_functions={'White Blood Cells': convert_white_blood_cells},
                       has_disease_col='has_sepsis',
                       subject_id_col='subject_id',
                       valueuom_col='valueuom',
                       scaler=MinMaxScaler(),
                       random_state=42,
                       test_size=0.2)

/Users/artemij/miniconda3/lib/python3.12/site-packages/sickness_screening/train_model.py:51: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_to_train_csv)


Accuracy: 0.6039456120661804
Classification Report:
               precision    recall  f1-score   support

       False       0.44      0.32      0.37    344329
        True       0.66      0.77      0.71    596615

    accuracy                           0.60    940944
   macro avg       0.55      0.54      0.54    940944
weighted avg       0.58      0.60      0.59    940944



В этой функции мы обучаем **RandomForestClassifier** из scikit-learn на наборе данных с одной категориальной колонкой, с одной числовой колонкой
и с одной категориальной колонкой, которую можно преобразовать в числовую. В качестве метода нормализации используется **MinMaxScaler** из scikit-learn.

#### Например, можно вставить такие модели, как CatBoostClassifier или SVC с разными ядрами
CatBoostClassifier:

In [ ]:
class_weights = {0: 1, 1: 15}
clf = CatBoostClassifier(loss_function='MultiClassOneVsAll', class_weights=class_weights, iterations=50, learning_rate=0.1, depth=5)
clf.fit(X_train, y_train)

SVC с использованием гауссова ядра с радиальной базовой функцией (RBF):

In [ ]:
class_weights = {0: 1, 1: 13}
param_dist = {
    'C': reciprocal(0.1, 100),
    'gamma': reciprocal(0.01, 10),
    'kernel': ['rbf']
}

svm_model = SVC(class_weight=class_weights, random_state=42)
random_search = RandomizedSearchCV(
    svm_model,
    param_distributions=param_dist,
    n_iter=10,
    cv=5,
    scoring=make_scorer(recall_score, pos_label=1),
    n_jobs=-1
)